## MNIST Training with MXNet and Gluon

MNIST is a widely used dataset for handwritten digit classification. It consists of 70,000 labeled 28x28 pixel grayscale images of hand-written digits. The dataset is split into 60,000 training images and 10,000 test images. There are 10 classes (one for each of the 10 digits). This tutorial will show how to train and test an MNIST model on SageMaker using MXNet and the Gluon API.



In [ ]:
!/bin/bash ./utils/setup.sh

In [ ]:
import os
import boto3
import sagemaker
# from sagemaker.mxnet import MXNet
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

role = get_execution_role()

## Download training and test data

In [ ]:
WORK_DIRECTORY = '/home/ec2-user/SageMaker/gluonts_sagemaker/data'

## Uploading the data

We use the `sagemaker.Session.upload_data` function to upload our datasets to an S3 location. The return value `inputs` identifies the location -- we will use this later when we start the training job.

In [ ]:
inputs = sagemaker_session.upload_data(path=WORK_DIRECTORY, key_prefix='data/gluonts')
print(inputs)

## Implement the training function

We need to provide a training script that can run on the SageMaker platform. The training scripts are essentially the same as one you would write for local training, except that you need to provide a `train` function. The `train` function will check for the validation accuracy at the end of every epoch and checkpoints the best model so far, along with the optimizer state, in the folder `/opt/ml/checkpoints` if the folder path exists, else it will skip the checkpointing. When SageMaker calls your function, it will pass in arguments that describe the training environment. Check the script below to see how this works.

The script here is an adaptation of the [Gluon MNIST example](https://github.com/apache/incubator-mxnet/blob/master/example/gluon/mnist.py) provided by the [Apache MXNet](https://mxnet.incubator.apache.org/) project. 

In [ ]:
# !cat 'train.py'

## Run the training script on SageMaker

The ```MXNet``` class allows us to run our training function on SageMaker infrastructure. We need to configure it with our training script, an IAM role, the number of training instances, and the training instance type. In this case we will run our training job on a single c4.xlarge instance. 

In [ ]:
git_config = {'repo': 'https://github.com/whn09/gluonts_sagemaker.git', 'branch': 'main'}

hyperparameters = {'algo-name': 'DeepAR', 'freq': '1D', 'prediction-length': 5, 'context-length': 5*10, 'epochs': 2, 'batch-size': 32, 'num-batches-per-epoch': 50}
# hyperparameters = {'algo-name': 'NPTS', 'freq': '1D', 'prediction-length': 5, 'context-length': 5*10, 'epochs': 2, 'batch-size': 32, 'num-batches-per-epoch': 50}
# hyperparameters = {'algo-name': 'SeasonalNaive', 'freq': '1D', 'prediction-length': 5, 'context-length': 5*10, 'epochs': 2, 'batch-size': 32, 'num-batches-per-epoch': 50}

m = PyTorch('train.py',  # MXNet
          source_dir='.',
          git_config=git_config,
          role=role,
          instance_count=1,
          instance_type='ml.c5.2xlarge', # 'local', 'ml.c5.2xlarge'
#           image_uri='579019700964.dkr.ecr.us-east-1.amazonaws.com/gluonts_sagemaker:latest',  # support R
          framework_version='1.9.1',  # '1.8.0'， '1.9.1'
          py_version='py38',  # 'py37', 'py38'
          hyperparameters=hyperparameters)

After we've constructed our `MXNet` object, we can fit it using the data we uploaded to S3. SageMaker makes sure our data is available in the local filesystem, so our training script can simply read the data from disk.


In [ ]:
m.fit({'train': inputs+'/train_1D.json', 'test': inputs+'/test_1D.json'})

In [ ]:
training_job_name = m.latest_training_job.name
# training_job_name = 'mxnet-training-2022-06-08-11-36-58-728'
print(training_job_name)
# m = MXNet.attach(training_job_name=training_job_name)

After training, we use the MXNet object to build and deploy an MXNetPredictor object. This creates a SageMaker endpoint that we can use to perform inference. 

This allows us to perform inference on json encoded multi-dimensional arrays. 

In [ ]:
# !rm -rf model.tar.gz
# !aws s3 cp s3://$bucket/$training_job_name/output/model.tar.gz .
# !tar -xvf model.tar.gz

In [ ]:
# !rm -rf model-gluonts.tar.gz
# !rm -rf model/DeepAR
# !mv DeepAR model/
# !cd model && tar -czvf ../model-gluonts.tar.gz *

# !aws s3 cp model-gluonts.tar.gz s3://$bucket/$training_job_name/output/model-gluonts.tar.gz

In [ ]:
predictor = m.deploy(initial_instance_count=1, instance_type='ml.m5.xlarge')  # 'local', 'ml.m5.xlarge'

# # BYOM (optional)
# # from sagemaker.mxnet.model import MXNetModel
# from sagemaker.pytorch.model import PyTorchModel
# model = PyTorchModel(model_data='s3://{}/{}/output/model-gluonts.tar.gz'.format(bucket, training_job_name),  # MXNetModel
#                          role=role, 
#                          entry_point='inference.py',
# #                          image_uri="579019700964.dkr.ecr.us-east-1.amazonaws.com/gluonts_sagemaker:latest",  # support R
#                          framework_version='1.9.1',  # '1.8.0', '1.9.1'
#                          py_version='py38')  # 'py37', 'py38'
# predictor = model.deploy(instance_type='ml.m5.xlarge', initial_instance_count=1)  # 'local', 'ml.m5.xlarge'

In [ ]:
!head -n 1 /home/ec2-user/SageMaker/gluonts_sagemaker/data/test_1D.json

We can now use this predictor to classify hand-written digits. Drawing into the image box loads the pixel data into a 'data' variable in this notebook, which we can then pass to the mxnet predictor. 

In [ ]:
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

test_data = {"start": "2020-01-22 00:00:00", "target": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 4.0, 4.0, 5.0, 7.0, 7.0, 7.0, 11.0, 16.0, 21.0, 22.0, 22.0, 22.0, 24.0, 24.0, 40.0, 40.0, 74.0, 84.0, 94.0, 110.0, 110.0, 120.0, 170.0, 174.0, 237.0], "id": "Afghanistan_"}
test_data['target'] = test_data['target'][:-5]

data = {'instances': [test_data]}
data['freq'] = '1D'
data['target_quantile'] = 0.5

predictor.serializer = JSONSerializer()
predictor.deserializer = JSONDeserializer()

The predictor runs inference on our input data and returns the predicted digit (as a float value, so we convert to int for display).

In [ ]:
result = predictor.predict(data)
print(result)

## Cleanup

After you have finished with this example, remember to delete the prediction endpoint to release the instance(s) associated with it.

In [ ]:
predictor.delete_endpoint()

## Batch Transform

In [ ]:
# transformer = m.transformer(
#     instance_count=1,
#     instance_type="ml.m5.xlarge",
#     strategy="SingleRecord",
#     assemble_with="Line",
#     output_path=inputs+'/output',
# )

# test_s3 = inputs + '/test_1D.json'

# transformer.transform(test_s3, split_type="Line", wait=True)